# MFIN7037 Final Replication: Gross Profitability Premium

Group Members: 
> LI Zhaozhen (3035875428 Section A)

> SHI Qimeng (3035873781 Section A)

> WANG Yanyu (3035875777 Section B)

Paper replicated:
> Novy-Marx, R. (2010, April 29). *The other side of value: Good growth and the Gross Profitability Premium*. NBER. Retrieved May 20, 2022, from https://www.nber.org/papers/w15940. 

# Data Collection & Cleaning

We collected yearly accounting data from Compustat and merged it with comp_finratios and msf_delisting_adjusted_with_lags to get the final dataset for our replication.

1. Collect yearly accounting data from Compustat
* Already impose the condition that companies must be listed in NYSE, AMEX, NASDAQ (exchg = 11, 12, 14)
* Main accounting variables:
  - GP: gross profits = REVT - COGS
  - IB: earnings before extraordinary items
  - FCF: free cashflow = NI + DP - WCAPCH - CAPX
  - AT: assets

2. Merge with comp_finratios (using gvkey)
* **Lag accounting data to the end of June of the following year**
* Main variables from comp_finratios:
  - bm: book-to-market ratio
  - mktcap: size
  - ret_crsp: return
  - ffi49: Fama-French 49 industry
* exclude financial firms: ffi49 != 45,46,48 where
   - 45: Banks
   - 46: Insurance
   - 48: Trading

3. msf_delisting_adjusted_with_lags is used to filter out common stocks with shrcd = 10, 11


4. The final time period of our dataset: **1970/07/31 - 2021/12/31**
   - The 2010 paper covers 1963 to 2009. Yet, we are not able to get accounting data before 1970.
   - We extend the period to 2021 to see how the strategy performs in recent years.

In [2]:
import pandas as pd
import numpy as np
import urllib
import statsmodels.api as sm # statistical models including regression
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import os
import gc # garbage collector
from scipy.stats.mstats import winsorize
from tqdm import tqdm, trange
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"  # multiple output per jupyter notebook code block
%matplotlib inline

# path = r'C:\Users\yanyu\Desktop\quant\final\dataset'  # set path
#path =  r'C:\Users\zhaoz\Desktop\Quant Trading\dataset'
path =  '/Users/shiqimeng/Desktop/MFIN7037 QT/Final'

In [3]:
# Read finnancial ratio data, monthly frequency (starts from 1970)
comp_finratios = pd.read_parquet(path + os.sep + 'comp_finratios.parquet')
comp_finratios.rename(columns = {'public_date': 'date','ret_crsp':'ret','gvkey':'gvkey_s'},  inplace = True) # rename public_date to date
comp_finratios.date = comp_finratios.date.map(lambda x: int(''.join(filter(str.isdigit, x)))) # convert date strings into integers for filtering
comp_finratios['gvkey'] = comp_finratios['gvkey_s'].apply(lambda x: str(x).zfill(6))

In [4]:
# Full sample
comp_finratios = comp_finratios[(comp_finratios.date > 19630000) & # data after 1963
                (comp_finratios.ffi49 != (45|46|48))] # exclude financial firms

# Select the columns needed
comp_finratios = comp_finratios[['cusip','gvkey','permno','date','bm','mktcap','ret','price','ffi49']]

comp_finratios.head(10)
print('number of rows in raw data: ', len(comp_finratios))
print('This is the date range:')
min(comp_finratios['date']), max(comp_finratios['date'])

,cusip,gvkey,permno,date,bm,mktcap,ret,price,ffi49
0,25388K10,132599,87763.0,20061130,0.477322,936.038174,0.024621,10.82000,34.0
1,25388K10,132599,87763.0,20061231,0.477322,1160.099087,0.239372,13.41000,34.0
2,25388K10,132599,87763.0,20070131,0.477322,1166.154760,0.005220,13.48000,34.0
3,46262810,132600,87783.0,20000531,0.089041,112.237500,-0.210526,3.75000,34.0
4,46262810,132600,87783.0,20000630,0.089041,147.059937,0.308333,4.90625,34.0
5,46262810,132600,87783.0,20000930,0.287903,105.210000,0.000000,3.50000,34.0
6,46262810,132600,87783.0,20001031,0.287903,45.090000,-0.571429,1.50000,34.0
7,46262810,132600,87783.0,20001130,0.340918,45.090000,0.000000,1.50000,34.0
8,46262810,132600,87783.0,20001231,0.340918,21.601312,-0.520833,0.71875,34.0
9,46262810,132600,87783.0,20010131,0.340918,26.383000,0.217391,0.87500,34.0


number of rows in raw data:  2707472
This is the date range:


(19700131, 20211231)

In [5]:
# Create log(bm),log(mktcap),r(1,0),r(12,2)
comp_finratios['log_bm'] = np.log(comp_finratios['bm'])
comp_finratios['log_size'] = np.log(comp_finratios['mktcap'])
comp_finratios['ret_1_0'] = comp_finratios.sort_values(['date']).groupby('permno')['ret'].shift(periods = 1)
comp_finratios['mktcap_lag1'] = comp_finratios.sort_values(['date']).groupby('permno')['mktcap'].shift(periods = 1)

# Calculate rolling 10-month return
def rolling_prod(a, n=10) :
    ret = np.cumprod(a.values)
    ret[n:] = ret[n:] / ret[:-n]
    ret[:n-1] = np.nan
    return pd.Series(ret, index=a.index)

comp_finratios['ret_rolling_10'] = ((
    comp_finratios
    .assign(ret=(comp_finratios['ret'].fillna(0)+1))
    .sort_values(['date'])
    .groupby('permno')['ret']
    .apply(rolling_prod)) - 1)

comp_finratios['ret_12_2'] = comp_finratios.sort_values(['date']).groupby('permno')['ret_rolling_10'].shift(periods = 2)

In [6]:
# Check
grouped_df = comp_finratios.sort_values(['date']).groupby('permno')
print(grouped_df.get_group(87763.0), "\n\n")

            cusip   gvkey   permno      date        bm       mktcap       ret  \
2162406  25388K10  132599  87763.0  20000531  0.178543   699.879375 -0.210526   
2162407  25388K10  132599  87763.0  20000630  0.178543   938.572312  0.338462   
2162408  25388K10  132599  87763.0  20000731  0.178543  1301.774625  0.386973   
2162409  25388K10  132599  87763.0  20000831  0.267910   791.133750 -0.392265   
2162410  25388K10  132599  87763.0  20000930  0.267910  1022.329000  0.290909   
...           ...     ...      ...       ...       ...          ...       ...   
2162486  25388K10  132599  87763.0  20060930  0.404852   841.547970  0.076063   
2162487  25388K10  132599  87763.0  20061031  0.404852   923.778277  0.097713   
0        25388K10  132599  87763.0  20061130  0.477322   936.038174  0.024621   
1        25388K10  132599  87763.0  20061231  0.477322  1160.099087  0.239372   
2        25388K10  132599  87763.0  20070131  0.477322  1166.154760  0.005220   

           price  ffi49    

In [7]:
# Drop ret_rolling_10
comp_finratios = comp_finratios.drop(columns=['ret_rolling_10'])
# Free up unused objects from memory
gc.collect()

13

In [8]:
comp_finratios.head(10)
print('number of rows in raw data: ', len(comp_finratios))

,cusip,gvkey,permno,date,bm,mktcap,ret,price,ffi49,log_bm,log_size,ret_1_0,mktcap_lag1,ret_12_2
0,25388K10,132599,87763.0,20061130,0.477322,936.038174,0.024621,10.82000,34.0,-0.739565,6.841656,0.097713,923.778277,-0.230400
1,25388K10,132599,87763.0,20061231,0.477322,1160.099087,0.239372,13.41000,34.0,-0.739565,7.056261,0.024621,936.038174,-0.156550
2,25388K10,132599,87763.0,20070131,0.477322,1166.154760,0.005220,13.48000,34.0,-0.739565,7.061467,0.239372,1160.099087,-0.173415
3,46262810,132600,87783.0,20000531,0.089041,112.237500,-0.210526,3.75000,34.0,-2.418662,4.720617,NaN,NaN,NaN
4,46262810,132600,87783.0,20000630,0.089041,147.059937,0.308333,4.90625,34.0,-2.418662,4.990840,-0.210526,112.237500,NaN
5,46262810,132600,87783.0,20000930,0.287903,105.210000,0.000000,3.50000,34.0,-1.245132,4.655958,-0.295597,104.909000,NaN
6,46262810,132600,87783.0,20001031,0.287903,45.090000,-0.571429,1.50000,34.0,-1.245132,3.808660,0.000000,105.210000,NaN
7,46262810,132600,87783.0,20001130,0.340918,45.090000,0.000000,1.50000,34.0,-1.076113,3.808660,-0.571429,45.090000,NaN
8,46262810,132600,87783.0,20001231,0.340918,21.601312,-0.520833,0.71875,34.0,-1.076113,3.072754,0.000000,45.090000,NaN
9,46262810,132600,87783.0,20010131,0.340918,26.383000,0.217391,0.87500,34.0,-1.076113,3.272720,-0.520833,21.601312,NaN


number of rows in raw data:  2707472


In [9]:
comp_finratios['fyear'] = comp_finratios['date'].apply(lambda x: int(str(x)[:4]))

In [10]:
# Read fundamentals data
fundamentals = pd.read_parquet(path + os.sep + 'fundamentals.parquet')
fundamentals.rename(columns = {'datadate': 'date','gvkey':'gvkey_s'},  inplace = True) # Rename datadate to date
fundamentals['gvkey'] = fundamentals['gvkey_s'].apply(lambda x: str(x).zfill(6))

# Calculate FCF = NI + DP - WCAPCH - CAPX
# Fill NA, must have ni
fundamentals['dp'] = fundamentals['dp'].fillna(0)
fundamentals['wcapch'] = fundamentals['wcapch'].fillna(0)
fundamentals['capx'] = fundamentals['capx'].fillna(0)
fundamentals['fcf'] = fundamentals.apply(lambda x: x.ni + x.dp - x.wcapch - x.capx, axis  = 1)

# Calculate gross profit
fundamentals['gp'] = fundamentals.apply(lambda x: x.revt - x.cogs, axis  = 1)

fundamentals = fundamentals[
    (fundamentals.gp.notnull() & # has gp
     fundamentals.ib.notnull() & # has IB
     fundamentals.fcf.notnull() & # has fcf
    pd.notnull(fundamentals['at'])) # has asset
]    

# Select columns needed
fundamentals = fundamentals[['gvkey','fyear','at','gp','ib','fcf','exchg']]

fundamentals.head(10)
print('number of rows in raw data: ', len(fundamentals))
print('This is the year range:')
min(fundamentals['fyear']), max(fundamentals['fyear'])

,gvkey,fyear,at,gp,ib,fcf,exchg
1,001000,1964,1.416,0.558,0.039,0.105,12
2,001000,1965,2.310,0.346,-0.197,-0.115,12
3,001000,1966,2.430,1.204,0.201,0.221,12
4,001000,1967,2.456,0.893,-0.090,-0.072,12
5,001000,1968,5.922,2.345,0.347,-1.680,12
6,001000,1969,28.712,12.206,1.835,-2.144,12
7,001000,1970,33.450,14.806,1.878,-0.857,12
8,001000,1971,29.330,13.060,0.138,-12.115,12
9,001000,1972,19.907,11.660,1.554,6.079,12
10,001000,1973,21.771,13.046,1.863,0.475,12


number of rows in raw data:  238412
This is the year range:


(1962, 2021)

In [11]:
# Scale gp, ib, fcf by at
fundamentals['gp_at'] = fundamentals['gp'] / fundamentals['at'] 
fundamentals['ib_at'] = fundamentals['ib'] / fundamentals['at'] 
fundamentals['fcf_at'] = fundamentals['fcf'] / fundamentals['at'] 

In [12]:
# Add ffi49 to fundamentals
df_49 = comp_finratios[['gvkey','fyear','date','ffi49']]
df_49 = df_49.drop_duplicates(['gvkey','fyear'])
df_49 = df_49[['gvkey','fyear','ffi49']].set_index(['gvkey','fyear'])
fundamentals = fundamentals.join(df_49, on = ['gvkey','fyear'], how="left")
fundamentals.head()

,gvkey,fyear,at,gp,ib,fcf,exchg,gp_at,ib_at,fcf_at,ffi49
1,001000,1964,1.416,0.558,0.039,0.105,12,0.394068,0.027542,0.074153,NaN
2,001000,1965,2.310,0.346,-0.197,-0.115,12,0.149784,-0.085281,-0.049784,NaN
3,001000,1966,2.430,1.204,0.201,0.221,12,0.495473,0.082716,0.090947,NaN
4,001000,1967,2.456,0.893,-0.090,-0.072,12,0.363599,-0.036645,-0.029316,NaN
5,001000,1968,5.922,2.345,0.347,-1.680,12,0.395981,0.058595,-0.283688,NaN


In [13]:
# Demeaned profitability measures (gp_at,ib_at,fcf_at)
dm = fundamentals.groupby(['fyear','ffi49']).agg('mean')
dm.rename(columns = {'gp_at': 'gp_at_m','ib_at':'ib_at_m','fcf_at':'fcf_at_m'},  inplace = True) 
fundamentals = fundamentals.join(dm[['gp_at_m','ib_at_m','fcf_at_m']], on = ['fyear','ffi49'], how="left")

fundamentals['gp_at_dm'] = fundamentals['gp_at'] - fundamentals['gp_at_m']
fundamentals['ib_at_dm'] = fundamentals['ib_at'] - fundamentals['ib_at_m']
fundamentals['fcf_at_dm'] = fundamentals['fcf_at'] - fundamentals['fcf_at_m'] 

# Drop mean columns
fundamentals = fundamentals.drop(columns=['gp_at_m','ib_at_m','fcf_at_m','ffi49'])

fundamentals

,gvkey,fyear,at,gp,ib,fcf,exchg,gp_at,ib_at,fcf_at,gp_at_dm,ib_at_dm,fcf_at_dm
1,001000,1964,1.416,0.558,0.039,0.105,12,0.394068,0.027542,0.074153,NaN,NaN,NaN
2,001000,1965,2.310,0.346,-0.197,-0.115,12,0.149784,-0.085281,-0.049784,NaN,NaN,NaN
3,001000,1966,2.430,1.204,0.201,0.221,12,0.495473,0.082716,0.090947,NaN,NaN,NaN
4,001000,1967,2.456,0.893,-0.090,-0.072,12,0.363599,-0.036645,-0.029316,NaN,NaN,NaN
5,001000,1968,5.922,2.345,0.347,-1.680,12,0.395981,0.058595,-0.283688,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308080,345980,2021,1283.000,1117.000,-361.000,-354.000,14,0.870616,-0.281372,-0.275916,0.332494,-0.318475,-0.311553
308083,347007,2021,468.910,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN
308084,347085,2018,108.754,102.881,16.851,3.797,14,0.945997,0.154946,0.034914,NaN,NaN,NaN
308085,347085,2019,117.551,105.934,18.467,12.545,14,0.901175,0.157098,0.106720,NaN,NaN,NaN


In [14]:
print('number of rows in comp_finratios data: ', len(comp_finratios))
print('number of rows in fundamentals data: ', len(fundamentals))

number of rows in comp_finratios data:  2707472
number of rows in fundamentals data:  238412


In [15]:
# Create a column to record the merge year
def merge_y(a):
    b = int(str(a)[:4])
    if int(str(a)[4:6]) < 7:
        b = int(str(a)[:4]) - 1
    return b

comp_finratios['merge_y'] = comp_finratios['date'].apply(merge_y)

# Merge
all_df = pd.merge(comp_finratios, fundamentals, left_on = ['gvkey','merge_y'],right_on = ['gvkey','fyear'], how="right")

all_df
print('number of rows in all_df data: ', len(all_df))

,cusip,gvkey,permno,date,bm,mktcap,ret,price,ffi49,log_bm,...,gp,ib,fcf,exchg,gp_at,ib_at,fcf_at,gp_at_dm,ib_at_dm,fcf_at_dm
0,NaN,001000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.558,0.039,0.105,12,0.394068,0.027542,0.074153,NaN,NaN,NaN
1,NaN,001000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.346,-0.197,-0.115,12,0.149784,-0.085281,-0.049784,NaN,NaN,NaN
2,NaN,001000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.204,0.201,0.221,12,0.495473,0.082716,0.090947,NaN,NaN,NaN
3,NaN,001000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.893,-0.090,-0.072,12,0.363599,-0.036645,-0.029316,NaN,NaN,NaN
4,NaN,001000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.345,0.347,-1.680,12,0.395981,0.058595,-0.283688,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043908,45256X10,347007,15533.0,20211130.0,NaN,3097.265035,-0.005109,7.79,NaN,NaN,...,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN
2043909,45256X10,347007,15533.0,20211231.0,NaN,2417.377570,-0.219512,6.08,NaN,NaN,...,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN
2043910,NaN,347085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102.881,16.851,3.797,14,0.945997,0.154946,0.034914,NaN,NaN,NaN
2043911,NaN,347085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,105.934,18.467,12.545,14,0.901175,0.157098,0.106720,NaN,NaN,NaN


number of rows in all_df data:  2043913


In [16]:
all_df.columns

Index(['cusip', 'gvkey', 'permno', 'date', 'bm', 'mktcap', 'ret', 'price',
       'ffi49', 'log_bm', 'log_size', 'ret_1_0', 'mktcap_lag1', 'ret_12_2',
       'fyear_x', 'merge_y', 'fyear_y', 'at', 'gp', 'ib', 'fcf', 'exchg',
       'gp_at', 'ib_at', 'fcf_at', 'gp_at_dm', 'ib_at_dm', 'fcf_at_dm'],
      dtype='object')

In [17]:
all_df = all_df.drop(columns=['merge_y', 'fyear_y'])
all_df.rename(columns = {'fyear_x': 'fyear'},  inplace = True)
all_df = all_df[
    pd.notnull(all_df['date']) # has date
]    
all_df
print('number of rows in all_df data: ', len(all_df))

,cusip,gvkey,permno,date,bm,mktcap,ret,price,ffi49,log_bm,...,gp,ib,fcf,exchg,gp_at,ib_at,fcf_at,gp_at_dm,ib_at_dm,fcf_at_dm
6,None,001000,25881.0,19710131.0,0.427328,23.895000,-0.100000,9.000,15.0,-0.850204,...,14.806,1.878,-0.857,12,0.442631,0.056143,-0.025620,NaN,NaN,NaN
7,00003210,001000,25881.0,19710228.0,0.431071,29.536875,0.236111,11.125,15.0,-0.841482,...,14.806,1.878,-0.857,12,0.442631,0.056143,-0.025620,NaN,NaN,NaN
8,00003210,001000,25881.0,19710331.0,0.431071,25.886250,-0.123596,9.750,15.0,-0.841482,...,14.806,1.878,-0.857,12,0.442631,0.056143,-0.025620,NaN,NaN,NaN
9,00003210,001000,25881.0,19710430.0,0.431071,28.541250,0.102564,10.750,15.0,-0.841482,...,14.806,1.878,-0.857,12,0.442631,0.056143,-0.025620,NaN,NaN,NaN
10,00003210,001000,25881.0,19710531.0,0.431071,22.899375,-0.197674,8.625,15.0,-0.841482,...,14.806,1.878,-0.857,12,0.442631,0.056143,-0.025620,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043905,45256X10,347007,15533.0,20210831.0,NaN,4451.366705,0.038321,11.380,NaN,NaN,...,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN
2043906,45256X10,347007,15533.0,20210930.0,NaN,3842.527310,-0.144113,9.740,NaN,NaN,...,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN
2043907,45256X10,347007,15533.0,20211031.0,NaN,3089.013270,-0.196099,7.830,NaN,NaN,...,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN
2043908,45256X10,347007,15533.0,20211130.0,NaN,3097.265035,-0.005109,7.790,NaN,NaN,...,-303.186,-346.790,-366.115,14,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN


number of rows in all_df data:  1978186


In [18]:
# Winsorize all independent vars
invar = ['gp_at','ib_at','fcf_at','gp_at_dm','ib_at_dm','fcf_at_dm','log_bm','log_size','ret_1_0','ret_12_2']

def wins(a) :
    w = winsorize(a.values, limits = [0.01,0.01])
    return pd.Series(w, index=a.index)

for i in invar:
    name = i + '_w'
    a = all_df.groupby('fyear')[i]
    all_df[name] = (all_df
                         .groupby('fyear')[i]
                         .apply(wins))
all_df

,cusip,gvkey,permno,date,bm,mktcap,ret,price,ffi49,log_bm,...,gp_at_w,ib_at_w,fcf_at_w,gp_at_dm_w,ib_at_dm_w,fcf_at_dm_w,log_bm_w,log_size_w,ret_1_0_w,ret_12_2_w
6,None,001000,25881.0,19710131.0,0.427328,23.895000,-0.100000,9.000,15.0,-0.850204,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.850204,3.173669,0.564286,NaN
7,00003210,001000,25881.0,19710228.0,0.431071,29.536875,0.236111,11.125,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.385639,-0.100000,NaN
8,00003210,001000,25881.0,19710331.0,0.431071,25.886250,-0.123596,9.750,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.253712,0.236111,NaN
9,00003210,001000,25881.0,19710430.0,0.431071,28.541250,0.102564,10.750,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.351350,-0.123596,NaN
10,00003210,001000,25881.0,19710531.0,0.431071,22.899375,-0.197674,8.625,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.131110,0.102564,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043905,45256X10,347007,15533.0,20210831.0,NaN,4451.366705,0.038321,11.380,NaN,NaN,...,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN,NaN,8.400966,-0.232493,-0.079604
2043906,45256X10,347007,15533.0,20210930.0,NaN,3842.527310,-0.144113,9.740,NaN,NaN,...,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN,NaN,8.253886,-0.144113,-0.673397
2043907,45256X10,347007,15533.0,20211031.0,NaN,3089.013270,-0.196099,7.830,NaN,NaN,...,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN,NaN,8.035607,-0.144113,-0.673397
2043908,45256X10,347007,15533.0,20211130.0,NaN,3097.265035,-0.005109,7.790,NaN,NaN,...,-0.646576,-0.739566,-0.780779,NaN,NaN,NaN,NaN,8.038275,-0.196099,-0.673397


In [19]:
# Free up unused objects from memory
gc.collect()

0

In [20]:
# Read msf (get shrcd only)
msf = pd.read_parquet(path + os.sep + 'msf_delisting_adjusted_with_lags.parquet')

In [21]:
msf = msf[msf['shrcd'].isin(set([10,11]))]  # filter out those with share code 10, 11
msf = msf[['permno']]
msf = msf.drop_duplicates()

In [22]:
final_df = pd.merge(all_df, msf, on = ['permno'], how = 'inner')
final_df

,cusip,gvkey,permno,date,bm,mktcap,ret,price,ffi49,log_bm,...,gp_at_w,ib_at_w,fcf_at_w,gp_at_dm_w,ib_at_dm_w,fcf_at_dm_w,log_bm_w,log_size_w,ret_1_0_w,ret_12_2_w
0,None,001000,25881.0,19710131.0,0.427328,23.895000,-0.100000,9.000,15.0,-0.850204,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.850204,3.173669,0.564286,NaN
1,00003210,001000,25881.0,19710228.0,0.431071,29.536875,0.236111,11.125,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.385639,-0.100000,NaN
2,00003210,001000,25881.0,19710331.0,0.431071,25.886250,-0.123596,9.750,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.253712,0.236111,NaN
3,00003210,001000,25881.0,19710430.0,0.431071,28.541250,0.102564,10.750,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.351350,-0.123596,NaN
4,00003210,001000,25881.0,19710531.0,0.431071,22.899375,-0.197674,8.625,15.0,-0.841482,...,0.442631,0.056143,-0.025620,1.177835,0.285122,1.132998,-0.841482,3.131110,0.102564,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978181,21077C10,345980,20333.0,20210831.0,0.104389,4023.180021,-0.274648,7.210,43.0,-2.259633,...,0.870616,-0.281372,-0.275916,0.332494,-0.318475,-0.311553,-2.259633,8.299828,-0.245254,NaN
1978182,21077C10,345980,20333.0,20210930.0,0.104389,3084.900022,-0.242718,5.460,43.0,-2.259633,...,0.870616,-0.281372,-0.275916,0.332494,-0.318475,-0.311553,-2.259633,8.034275,-0.274648,NaN
1978183,21077C10,345980,20333.0,20211031.0,0.104389,2942.699945,-0.065934,5.100,43.0,-2.259633,...,0.870616,-0.281372,-0.275916,0.332494,-0.318475,-0.311553,-2.259633,7.987083,-0.242718,NaN
1978184,21077C10,345980,20333.0,20211130.0,0.239105,2152.210011,-0.268627,3.730,43.0,-1.430854,...,0.870616,-0.281372,-0.275916,0.332494,-0.318475,-0.311553,-1.430854,7.674251,-0.065934,NaN


In [23]:
# Remove nan in ret_1_0,ret_12_2,ret,ffi49,bm,gp_at_dm,ib_at_dm,fcf_at_dm
final_df = final_df[
    (final_df.ret_1_0.notnull() & 
     final_df.ret_12_2.notnull() & 
     final_df.ret.notnull() & 
     final_df.ffi49.notnull() &
     final_df.bm.notnull() &
     final_df.gp_at_dm.notnull() &
     final_df.ib_at_dm.notnull() &
     final_df.fcf_at_dm.notnull()
    )]    

print('Number of observations {}'.format(len(final_df)))
print('This is the date range:')
min(final_df['date']), max(final_df['date'])

Number of observations 1726084
This is the date range:


(19700731.0, 20211231.0)

In [24]:
final_df.columns

Index(['cusip', 'gvkey', 'permno', 'date', 'bm', 'mktcap', 'ret', 'price',
       'ffi49', 'log_bm', 'log_size', 'ret_1_0', 'mktcap_lag1', 'ret_12_2',
       'fyear', 'at', 'gp', 'ib', 'fcf', 'exchg', 'gp_at', 'ib_at', 'fcf_at',
       'gp_at_dm', 'ib_at_dm', 'fcf_at_dm', 'gp_at_w', 'ib_at_w', 'fcf_at_w',
       'gp_at_dm_w', 'ib_at_dm_w', 'fcf_at_dm_w', 'log_bm_w', 'log_size_w',
       'ret_1_0_w', 'ret_12_2_w'],
      dtype='object')

In [25]:
# Save dataset
final_df.to_parquet('gp.parquet')

### The main variables used in the tables are summarized below:

<img src="dataset/var_summary.png" width = 90%/>

# Table 2

### Fama-MacBeth regressions of returns on measures of profitability

Panel A reports results from Fama-MacBeth regressions of firms’ returns on gross profits (revenues minus cost of goods sold, Compustat REVT - COGS), income before extraordinary items (IB), and free cashflow (net income plus amortization and depreciation minus changes in working capital minus capital expenditures, NI + DP - WCAPCH - CAPX), each scaled by assets (AT). 

Panel B repeats the tests of panel A, employing profitability measures demeaned by industry (Fama-French 49). Regressions include controls for book-to- market (log(bm)), size (log(me)), and past performance measured at horizons of one month ($r_{1,0}$) and twelve to two months ($r_{12,2}$). 

Independent variables are Winsorized at the one and 99% levels. 

The sample excludes financial firms (those with ffi49 = 45, 46, 48 - 45: Banks, 46: Insurance, 48: Trading), and covers **December 1970 to December 2021**.

In [3]:
# Read cleaned data
t2_df = pd.read_parquet(path + os.sep + 'gp.parquet')

In [27]:
# Show columns
t2_df.columns

Index(['cusip', 'gvkey', 'permno', 'date', 'bm', 'mktcap', 'ret', 'price',
       'ffi49', 'log_bm', 'log_size', 'ret_1_0', 'mktcap_lag1', 'ret_12_2',
       'fyear', 'at', 'gp', 'ib', 'fcf', 'exchg', 'gp_at', 'ib_at', 'fcf_at',
       'gp_at_dm', 'ib_at_dm', 'fcf_at_dm', 'gp_at_w', 'ib_at_w', 'fcf_at_w',
       'gp_at_dm_w', 'ib_at_dm_w', 'fcf_at_dm_w', 'log_bm_w', 'log_size_w',
       'ret_1_0_w', 'ret_12_2_w'],
      dtype='object')

#### Explanation for choosing the sample period: December 1970 to December 2021
The full sample period after we clean the data covers July 1970 to December 2021.

However, as shown in the code below, the first 5 months only have 2 securities. The number of observations are too small to carry a valid regression. Except for the first 5 months, all other months have over 1,000 observations, which is fine. Therefore, we are going to exclude the first 5 months.

In [28]:
a = t2_df.groupby('date')  # groupby date
r = a.apply(lambda x: len(x['permno'].values))
r.loc[lambda x: x<1000]

date
19700731.0    2
19700831.0    2
19700930.0    2
19701031.0    2
19701130.0    2
dtype: int64

In [29]:
# Define a function for Fama-MacBeth regression
# groupby date and run cross-sectional regression at each point of time
# the output is a time series of the value of coefficients
# report the average and standard deviation of the coefficients

from sklearn.linear_model import LinearRegression

def reg(a, voi) :
    '''
    input: a - groupby object
           voi - a list of the variables of interest in the regression
    '''
    # extract independent variables from a
    ind_vars = voi + ['log_bm_w', 'log_size_w','ret_1_0_w', 'ret_12_2_w']
    # voi + control varialbes 'log_bm_w', 'log_size_w','ret_1_0_w', 'ret_12_2_w'
    
    # create a dict to store results
    results = dict()
    
    for date in list(a.groups.keys())[5:]:
      reg_df = a.get_group(date)
      X = reg_df[ind_vars]
      y = reg_df['ret']
      reg = LinearRegression().fit(X, y)
      coef = reg.coef_
      results[date] = coef
    
    cols = []    # create column names
    for i in ind_vars:
        cols.append(i)
    
    # turn dict into df
    output_df = pd.DataFrame.from_dict(results, orient = 'index', columns = cols) 
    
    return output_df

In [30]:
# Define a funciton for showing the mean and t-stat
def mean_tstat(reg_df, reg_no):
    '''
    input: reg_df - df of the time series of coefficients for independent variables in the Fama-Macbeth regression
           regno - regression index
    output: mean (scaled by 100) and t-statistic for each independent variable
    '''
    sm = (reg_df.mean()*100).rename('mean_100_' + reg_no)   # scale the mean by 100
    st = (reg_df.mean()/(reg_df.std()/np.sqrt(len(reg_df)))).rename('t-stat_' + reg_no)
    reg_re = pd.concat([sm,st], axis = 1)
    return reg_re

In [31]:
# Table output
print('-'*20 + 'Panel A: straight profitability variables' + '-'*20)
reg1_df = reg(a, ['gp_at_w'])
reg1_re = mean_tstat(reg1_df, '1')

reg2_df = reg(a, ['ib_at_w'])
reg2_re = mean_tstat(reg2_df, '2')

reg3_df = reg(a, ['fcf_at_w'])
reg3_re = mean_tstat(reg3_df, '3')

reg4_df = reg(a, ['gp_at_w', 'ib_at_w'])
reg4_re = mean_tstat(reg4_df, '4')

reg5_df = reg(a, ['gp_at_w', 'fcf_at_w'])
reg5_re = mean_tstat(reg5_df, '5')

reg6_df = reg(a, ['ib_at_w', 'fcf_at_w'])
reg6_re = mean_tstat(reg6_df, '6')

reg7_df = reg(a, ['gp_at_w', 'ib_at_w', 'fcf_at_w'])
reg7_re = mean_tstat(reg7_df, '7')

panel_A = pd.concat([reg1_re, reg2_re, reg3_re, reg4_re, reg5_re, reg6_re, reg7_re], join='outer', axis=1)
panel_A.loc[['gp_at_w', 'ib_at_w', 'fcf_at_w','log_bm_w', 'log_size_w','ret_1_0_w', 'ret_12_2_w']]

print('-'*20 + 'Panel B: profitability variables demeaned by industry' + '-'*20)
reg1m_df = reg(a, ['gp_at_dm_w'])
reg1m_re = mean_tstat(reg1m_df, '1')

reg2m_df = reg(a, ['ib_at_dm_w'])
reg2m_re = mean_tstat(reg2m_df, '2')

reg3m_df = reg(a, ['fcf_at_dm_w'])
reg3m_re = mean_tstat(reg3m_df, '3')

reg4m_df = reg(a, ['gp_at_dm_w', 'ib_at_dm_w'])
reg4m_re = mean_tstat(reg4m_df, '4')

reg5m_df = reg(a, ['gp_at_dm_w', 'fcf_at_dm_w'])
reg5m_re = mean_tstat(reg5m_df, '5')

reg6m_df = reg(a, ['ib_at_dm_w', 'fcf_at_dm_w'])
reg6m_re = mean_tstat(reg6m_df, '6')

reg7m_df = reg(a, ['gp_at_dm_w', 'ib_at_dm_w', 'fcf_at_dm_w'])
reg7m_re = mean_tstat(reg7m_df, '7')

panel_B = pd.concat([reg1m_re, reg2m_re, reg3m_re, reg4m_re, reg5m_re, reg6m_re, reg7m_re], join='outer', axis=1)
panel_B.loc[['gp_at_dm_w', 'ib_at_dm_w', 'fcf_at_dm_w','log_bm_w', 'log_size_w','ret_1_0_w', 'ret_12_2_w']]

--------------------Panel A: straight profitability variables--------------------


,mean_100_1,t-stat_1,mean_100_2,t-stat_2,mean_100_3,t-stat_3,mean_100_4,t-stat_4,mean_100_5,t-stat_5,mean_100_6,t-stat_6,mean_100_7,t-stat_7
gp_at_w,1.386114,10.699561,NaN,NaN,NaN,NaN,1.374034,9.596088,1.505541,10.949904,NaN,NaN,1.328163,9.379494
ib_at_w,NaN,NaN,3.156711,6.184872,NaN,NaN,2.009187,3.651066,NaN,NaN,0.979775,1.517466,0.307898,0.464459
fcf_at_w,NaN,NaN,NaN,NaN,0.803842,2.702283,NaN,NaN,-0.092885,-0.292921,2.233161,6.046876,1.737552,4.866063
log_bm_w,0.760891,11.254003,0.731051,11.547033,0.625837,10.104754,0.869140,13.941343,0.792430,12.986015,0.724159,11.657926,0.860729,14.121838
log_size_w,0.312108,9.269813,0.280382,9.199428,0.284398,9.139777,0.322645,10.851240,0.327429,10.648854,0.283640,9.311710,0.324293,10.909818
ret_1_0_w,-5.250440,-13.273240,-5.452196,-13.925206,-5.285130,-13.575665,-5.592466,-14.387466,-5.430427,-14.039252,-5.485130,-14.054574,-5.620021,-14.505390
ret_12_2_w,0.237163,1.580834,0.167127,1.169179,0.239348,1.676888,0.201470,1.434321,0.261175,1.861935,0.150160,1.052194,0.186267,1.331079


--------------------Panel B: profitability variables demeaned by industry--------------------


,mean_100_1,t-stat_1,mean_100_2,t-stat_2,mean_100_3,t-stat_3,mean_100_4,t-stat_4,mean_100_5,t-stat_5,mean_100_6,t-stat_6,mean_100_7,t-stat_7
gp_at_dm_w,1.443540,13.178604,NaN,NaN,NaN,NaN,1.198780,11.928694,1.376348,13.731763,NaN,NaN,1.171905,11.770681
ib_at_dm_w,NaN,NaN,3.357920,7.839343,NaN,NaN,2.566983,5.830441,NaN,NaN,2.335604,4.707007,1.736825,3.438598
fcf_at_dm_w,NaN,NaN,NaN,NaN,1.129636,5.423962,NaN,NaN,0.608640,2.966598,1.100595,4.537286,0.899989,3.767440
log_bm_w,0.720738,10.369224,0.704976,10.480323,0.632846,9.582630,0.769004,11.086003,0.719655,10.495834,0.698972,10.443074,0.763191,11.055513
log_size_w,0.301707,8.829174,0.270226,8.436713,0.279055,8.552553,0.290465,9.006817,0.297996,9.043439,0.270150,8.442464,0.290084,9.000121
ret_1_0_w,-5.204059,-13.133924,-5.348518,-13.459399,-5.205246,-13.170966,-5.396086,-13.618665,-5.277841,-13.375094,-5.359631,-13.499847,-5.405939,-13.655306
ret_12_2_w,0.228061,1.514293,0.160275,1.081533,0.228076,1.548124,0.161502,1.091103,0.213842,1.453529,0.151479,1.020791,0.153354,1.034952


### Replication
<center>Paper</center> |<center>Replication</center>
- | - 
<img src="dataset/t2_paper.png" alt="paper table 2" width=87%/> | ![replicated table 2](dataset/t2_rep.png)

### Interpretation
In line with the results of the paper, in Panel A:
* Specification (1) shows that gross profitability (GP) has roughly the same power as book-to-market in predicting the cross-section of returns since the t-statistics are very close.
* As the coefficients of GP are all positive, profitable firms have higher average returns than unprofitable ones.
* Specification (2) & (3) show that earnings and free cashflow have power individually, but less than GP. Their t-stats are smaller than GP.
* Specification (4) & (5) show that GP subsumes earnings, and largely subsumes free cashflow. 
* Specification (6) shows that free cashflow subsumes earnings. 
* Specification (7) shows that free cashflow has incremental power above that in GP after controlling for earnings, but **GP is still the strongest predictive variable**. The t-stat for GP is the largest.

In Panel B:
* The tests tell the same story as panel A, but the t-stats are stronger.
* Specification (1) suggests that the predictive power of GP is stronger than value (log(BM)) and momentum ($r_{12,2}$).
* Compared with the paper period, **the power of the momentum strategy decreases over the years as the t-stats are not as large as before**.
* **The power of the gross profitability strategy becomes stronger as indicated by larger t-stats**.

# Table 3 

### Spearman rank correlations between independent variables
This table reports the **time-series averages** of the **cross-section Spearman rank correlations** between the independent variables employed in the Fama-MacBeth regressions of Table 2: 

<ul>
<li>gross profitability ((REVT - COGS)/AT)</li>
<li>earnings (IB/AT)</li>
<li>free cashflow ((NI + DP - WCAPCH - CAPX)/AT)</li>
<li>book-to-market</li>
<li>market equity</li>
<li>past performance measured at horizons of one month (r1;0) and twelve to two months (r12;2)</li>
</ul>

The sample excludes financial firms (those with one-digit SIC codes of six), and covers 1970 to 2021.

In [4]:
exclude = list(t2_df.groupby('date').groups.keys())[:5]
t2_df = t2_df[t2_df.date.apply(lambda x: x not in exclude)]  # exclude the first 5 months

In [14]:
#specify attributes to be tested
attributes = ['date','gp_at', 'ib_at', 'fcf_at', 'bm', 'mktcap', 'ret_1_0', 'ret_12_2']

#check if there is any missing value
t3_df = t2_df[(
    t2_df.gp_at.notnull() & # has GP/A
    t2_df.ib_at.notnull() & # has IB/A
    t2_df.fcf_at.notnull() & # has FCF/A
    t2_df.bm.notnull() & # has BM
    t2_df.mktcap.notnull() & #has ME
    t2_df.ret_1_0.notnull() & #has r(1,0)
    t2_df.ret_12_2.notnull() #has r(12,0)  
)][attributes].rename(columns = {'gp_at': 'gross profitability: GP/A', 
                                  'ib_at': 'earnings: IB/A', 
                                  'fcf_at': 'free cashflows: FCF/A',
                                 'bm': 'book-to-market: BM',
                                'mktcap': 'market equity: ME',
                                'ret_1_0': "prior month's performance r_1,0",
                                'ret_12_2': "prior year's performance r_12,2"}) 

# groupby date and find cross-section spearman correlation
t3_gp = t3_df.groupby('date')     
k1 = list(t3_gp.groups.keys())[0]
correlation_matrix = t3_gp.get_group(k1).corr(method = 'spearman').iloc[:-1, 1:]

for i in list(t3_gp.groups.keys())[1:]:
    new_matrix = t3_gp.get_group(i).corr(method = 'spearman').iloc[:-1, 1:]
    correlation_matrix = pd.concat([correlation_matrix, new_matrix], axis = 0)

# drop date and calculate time-series average
correlation_matrix = correlation_matrix.drop(['date'])
t3 = correlation_matrix.groupby(correlation_matrix.index).agg('mean').loc[[
    'gross profitability: GP/A','earnings: IB/A','free cashflows: FCF/A',
    'book-to-market: BM', 'market equity: ME',"prior month's performance r_1,0"
]]
t3

,gross profitability: GP/A,earnings: IB/A,free cashflows: FCF/A,book-to-market: BM,market equity: ME,"prior month's performance r_1,0","prior year's performance r_12,2"
gross profitability: GP/A,1.000000,0.475787,0.335547,-0.293140,-0.027871,0.017658,0.061866
earnings: IB/A,0.475787,1.000000,0.646441,-0.339219,0.320227,0.067557,0.251205
free cashflows: FCF/A,0.335547,0.646441,1.000000,-0.136213,0.216164,0.056510,0.183046
book-to-market: BM,-0.293140,-0.339219,-0.136213,1.000000,-0.307933,0.021611,-0.234671
market equity: ME,-0.027871,0.320227,0.216164,-0.307933,1.000000,0.079872,0.188822
"prior month's performance r_1,0",0.017658,0.067557,0.056510,0.021611,0.079872,1.000000,0.016100


### Replication
<center>Paper</center> 
<img src="dataset/t3_paper.jpg" alt="paper table 3" width=60%/>

### Interpretation

1. Earnings-related variables all positively correlated with each other.
2. Gross profitability and earnings are also negatively correlated with book-to-market, but weaker than to the negative correlation observed between book-to-market and size.
3. Earnings and free cashflows are positively associated with size (more profitable firms have higher market values), but correlation between gross profitability and size is negative. 
-> Strategies formed on the basis of gross profits-to-assets will be growth strategies, and relatively neutral with respect to size.

**Overall, Our correlation matrix shows the same pattern as correlation matrix in the original paper.**

# Table 4

### Excess returns to portfolios sorted on profitability.

This table shows **monthly value-weighted average excess returns** to portfolios sorted on **gross profits-to-assets[(REVT COGS)/AT]**, employing **NYSE breakpoints**, and results of **time series regressions** of these portfolios’ returns on the **Fama and French factors [the market factor (MKT), the size factor small-minus-large (SMB), and the value factor high-minus-low(HML)]** with test-statistics (in square brackets). It also shows **time series average portfolio characteristics [portfolio gross profits-to-assets(GP/A), book-to-market(B/M), average firm size (ME, in millions of dollars), and number of firms(n)]**. 

Panel B provides similar results for portfolios **sorted on book-to-market**.

The sample excludes financial firm (those with one-digit standard industrial classification codes of six) and covers **July 1970 to December 2021**.

In [15]:
t4_df = t2_df[['cusip', 'gvkey', 'permno', 'date', 'bm', 'mktcap', 'ret', 'log_bm', 'log_size', 'ret_1_0', 'mktcap_lag1', 'ret_12_2',  'exchg', 'gp_at', 'ib_at', 'fcf_at','gp','at']]

#import FF factors
ff4 = pd.read_parquet(path + os.sep + 'ff_four_factor_monthly.parquet')
ff4['date'] = (
    ff4
    .assign(date_tem=lambda df: pd.to_datetime(df['month_end']))
    .assign(date=
            lambda df: df['date_tem'].apply(lambda x: x.year) * 10000 
            + df['date_tem'].apply(lambda x: x.month)*100
            + df['date_tem'].apply(lambda x: x.day))
    ['date']
)
ff4 = ff4.drop(columns=['month_end'])
ff4.columns

Index(['mkt_rf', 'smb', 'hml', 'rf', 'mom', 'date'], dtype='object')

In [16]:
t4_df['gp_at_lag12'] = t4_df.sort_values(['date']).groupby('permno')['gp_at'].shift(periods = 1)

t4_df = pd.merge(
    t4_df,
    ff4[['date','rf']],
    on = 'date')

#create excess return - ret_rf
t4_df['ret_rf'] = t4_df['ret'] - t4_df['rf']

In [17]:
#seperate portfolios by NYSE breakpoints
#calculate NYSE breakpoints
def nyseBreakpoints(group, factor):
    nyse = group[(group.exchg == 11)].reset_index()  #NYSE stocks

    breakpoint1 = np.percentile(nyse[factor], 20, interpolation='midpoint')
    breakpoint2 = np.percentile(nyse[factor], 40, interpolation='midpoint')
    breakpoint3 = np.percentile(nyse[factor], 60, interpolation='midpoint')
    breakpoint4 = np.percentile(nyse[factor], 80, interpolation='midpoint')
    
    def sortOnFactor(factor):
        if factor <= breakpoint1:
            return 1
        elif ((factor > breakpoint1) & (factor <= breakpoint2)):
            return 2
        elif ((factor > breakpoint2) & (factor <= breakpoint3)):
            return 3
        elif ((factor > breakpoint3) & (factor <= breakpoint4)):
            return 4
        else:
            return 5
    
    group['bin_{}'.format(factor)] = group.apply(lambda x: sortOnFactor(x[factor]), axis = 1) 
    
    return group

In [18]:
#create bins 

#first group by date
t4_df_groups = t4_df.groupby('date')

#create empty dataframe to store sorted data
desired_columns = list(t4_df.columns)+['bin_gp_at']
t4_df_gp = pd.DataFrame(columns = desired_columns)

for date in tqdm(set(t4_df.date)):
    group = t4_df_groups.get_group(date)
    group = nyseBreakpoints(group, 'gp_at').reset_index()[desired_columns] #sort by gp_at 
    t4_df_gp = t4_df_gp.append(group, ignore_index = True)

100%|█████████████████████████████████████████| 613/613 [01:20<00:00,  7.64it/s]


In [24]:
#create bins 
#bins of BM 
#first group by date
t4_df_groups = t4_df.groupby('date')

#create empty dataframe to store sorted data
desired_columns = list(t4_df.columns)+['bin_bm']
t4_df_bm = pd.DataFrame(columns = desired_columns)

for date in tqdm(set(t4_df.date)):
    group = t4_df_groups.get_group(date)
    group = nyseBreakpoints(group, 'bm').reset_index()[desired_columns] #sort by bm
    t4_df_bm = t4_df_bm.append(group, ignore_index = True)

100%|█████████████████████████████████████████| 613/613 [01:20<00:00,  7.65it/s]


In [25]:
#create full_data with double sorted
t4_df_2 = pd.merge(
            t4_df_gp,
            t4_df_bm[['gvkey','date','bin_bm','permno']],
            on = ['date','gvkey','permno']
            ).reset_index()

In [26]:
t4_df_2.sort_values('date')

,index,cusip,gvkey,permno,date,bm,mktcap,ret,log_bm,log_size,...,gp_at,ib_at,fcf_at,gp,at,gp_at_lag12,rf,ret_rf,bin_gp_at,bin_bm
1278205,1278205,48917010,006386,44134.0,19701231.0,0.861815,76.409000,0.112245,-0.148715,4.336100,...,0.469971,0.094115,0.062412,34.955,74.377,NaN,0.004188,0.108057,4,4
1278020,1278020,34063910,004790,27748.0,19701231.0,0.803475,69.600000,0.230769,-0.218810,4.242765,...,0.003906,0.004964,0.004964,0.347,88.842,NaN,0.004188,0.226581,1,4
1278021,1278021,34069310,004791,46404.0,19701231.0,0.744238,131.601875,0.005208,-0.295394,4.879781,...,0.126024,0.028746,0.067898,45.563,361.541,NaN,0.004188,0.001020,1,3
1278022,1278022,34385610,004817,46164.0,19701231.0,0.876304,8.206625,0.596154,-0.132042,2.104942,...,0.699462,0.057710,0.044823,6.242,8.924,NaN,0.004188,0.591966,5,4
1278023,1278023,34482010,004830,47036.0,19701231.0,0.885442,23.644250,-0.008403,-0.121668,3.163120,...,1.372437,0.060017,0.016143,56.963,41.505,NaN,0.004188,-0.012592,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319950,1319950,03836N10,033643,17868.0,20211231.0,0.707088,180.801725,0.160870,-0.346600,5.197401,...,-0.643307,-0.651591,-0.646370,-73.934,114.928,-0.643307,0.000000,0.160870,1,4
1319949,1319949,46571Y10,033638,17887.0,20211231.0,0.390136,502.268830,0.221984,-0.941259,6.219135,...,0.255321,-0.006838,0.027651,166.418,651.800,0.255321,0.000000,0.221984,4,3
1319948,1319948,10948C10,033637,17867.0,20211231.0,0.912809,1481.215992,0.035294,-0.091228,7.300619,...,0.227174,0.014301,0.037713,735.500,3237.600,0.227174,0.000000,0.035294,3,5
1319957,1319957,19046P20,033695,17957.0,20211231.0,0.420917,651.428766,0.157558,-0.865319,6.479168,...,0.036569,0.010247,0.009865,96.377,2635.517,0.036569,0.000000,0.157558,1,3


In [27]:
#calculated valued weighted return for each bin
def calcu_return(df, factor):
    group = (
    df
    .groupby(['date', factor]) 
    .apply(
        lambda g: pd.Series({
            #'portfolio_vw': ((g['ret'] * g['mktcap_lag1']).sum() / g['mktcap_lag1'].sum()+1)**12-1 #value-weighted portfolio
            'portfolio_vw': (g['ret'] * g['mktcap_lag1']).sum() / g['mktcap_lag1'].sum() 
      })
  )
    ).reset_index()
    group = group.sort_values(['date', factor])
    return group

In [72]:
#L & S profitability strategy and Regression 
def ff_regression(portfolios, factor, n, t7 = False):
    grouped = portfolios.groupby(factor)
    result = pd.DataFrame()
    for i in range(1, n+1):
        #L & S strategy
        tem_portfolio = grouped.get_group(i)
        #Regression
        tem_portfolio = pd.merge(
            tem_portfolio,
            ff4,
            on='date' )
        
        if not t7:
            re = tem_portfolio.portfolio_vw.mean() * 100
        else:
            re = tem_portfolio.portfolio_vw.mean() * 100 / 12 # find monthly re for t5
        
        l1 = smf.ols('portfolio_vw ~  1 + mkt_rf + smb + hml', data = tem_portfolio ).fit()
        
        #print(sm.iolib.summary2.summary_col([l1], stars=True))

        result = result.append(pd.DataFrame(np.insert(l1.params.values,0,
                                                      [i, re])).T,
                                            ignore_index=True)

        result = result.append(pd.DataFrame(np.insert(['['+str(format(x, '.6f'))+']' for x in l1.tvalues.values], 0, #add square brackets for t-statistics
                                                      [i, ''])).T, ignore_index=True)

    result.columns = ['group', 're','alpha','MKT','SMB','HML']
    
    H_L = pd.DataFrame({
                'group': 'H-L',
                're': result.iloc[-2, 1] - result.iloc[0, 1],
                'alpha': result.iloc[-2, 2] - result.iloc[0, 2],
                'MKT': result.iloc[-2, 3] - result.iloc[0, 3],
                'SMB': result.iloc[-2, 4] - result.iloc[0, 4],
                'HML': result.iloc[-2, 5] - result.iloc[0, 5]},
                index = [0]) 
    
    result = result.append(H_L,ignore_index=True)

    return result

In [29]:
#portfolio characteristics which sorted on gross profits-to-assets
def chara_sort(df,factor):
    characteristics = (
        df[
            df['bm'].notnull() &
            df['mktcap'].notnull()&
            df['gp_at'].notnull()
        ]
        .groupby(['date', factor])[['gp_at', 'bm','mktcap','permno','gp','at']]
        .apply(
        lambda g: pd.Series({
            'GPA': (g['gp'].sum()/g['at'].sum()),
            'BM': ((g['mktcap']*g['bm']).sum()/g['mktcap'].sum()),
            'ME': (g['mktcap'].sum()/len(g['mktcap'])),  # mktcap in million
            'number of firm': round(g['permno'].nunique())  # count unique value
            })
        )
    .reset_index())
    
    characteristics_gp_ave = characteristics.groupby(factor)[['GPA', 'BM','ME','number of firm']].agg('mean')
    
    return characteristics_gp_ave

In [30]:
#calculated valued weighted return for each bin
portfolios_gp = calcu_return(t4_df_2,'bin_gp_at')

portfolios_gp.sort_values('date')

,date,bin_gp_at,portfolio_vw
0,19701231.0,1,0.077388
1,19701231.0,2,0.059762
2,19701231.0,3,0.056873
3,19701231.0,4,0.087079
4,19701231.0,5,0.045180
...,...,...,...
3062,20211231.0,3,0.046239
3063,20211231.0,4,0.053857
3060,20211231.0,1,0.021658
3061,20211231.0,2,0.047471


In [44]:
regre_gp = ff_regression(portfolios_gp,'bin_gp_at',5)
print('Fama-French Regression:')
regre_gp
print('Time-Series Average Portfolio Characteristics')
characteristics_gp = chara_sort(t4_df_2,'bin_gp_at')
characteristics_gp

Fama-French Regression:


,group,re,alpha,MKT,SMB,HML
0,1.0,0.90843,0.000531,1.104977,-0.027718,0.522898
1,1,,[0.658993],[59.306223],[-1.015931],[19.172011]
2,2.0,0.886646,0.001924,0.980722,-0.012833,0.232677
3,2,,[3.087041],[68.056621],[-0.608147],[11.030144]
4,3.0,1.040121,0.003808,1.000947,-0.054312,0.078903
5,3,,[6.074990],[69.050059],[-2.558564],[3.718354]
6,4.0,1.108241,0.005318,0.991943,0.014365,-0.214613
7,4,,[9.223519],[74.408450],[0.735840],[-10.997511]
8,5.0,1.172161,0.00662,0.92275,-0.109294,-0.248994
9,5,,[9.962102],[60.050795],[-4.857133],[-11.069456]


Time-Series Average Portfolio Characteristics


,GPA,BM,ME,number of firm
bin_gp_at,,,,
1,0.041765,24.597091,1790.890670,695.874388
2,0.159217,0.798075,2907.557555,411.504078
3,0.270181,0.587804,3216.477058,478.592170
4,0.396817,0.425538,3614.262401,557.349103
5,0.635983,0.281110,3309.745756,668.145188


Note: The unit for alpha value is decimal while re is in percentage.

### Replication
<center>Paper</center> 
<img src="dataset/t4_panel_a_paper.jpg" alt="paper table 4 panel A" width=60%/>

**Interpretation：**

*Portfolios are formed by independently quintile sorting on the two variables, using NYSE breaks. The sample excludes financial firms, and covers July 1963 to December 2009. The r is the gap between maxmimum return and minmimum return amoing each bin, the t value is filled with zeros. For the regression table, even rows are regression coefficients, odd rows are T values. Each two lines represents a bin.*

Comparing with paper:
 
2. The regression shows gross profits-to-assets portfolios’ average excess returns are generally increasing with profitability, significant profitable-minus-unprofitable return spread is observed.
3. Profitable firms tend to be growth firms, while the unprofitable firms tend to be value firms
4. The power that gross profits-to-assets has predicting the cross section of average returns is economically as well as statistically significant. By analyzing portfolios double sorted on SM and traded by profitability, this section shows that its power is economically significant even among the most profitable stocks.
5. The negative correlation between profitability and book-to-market observed in Table 4 suggests that the performance of value strategies can be improved by controlling for profitability,and that the performance of profitability strategies can be improved by controlling for book-to-market. 
6. The characteristics of Time-Series Average Portfolio are similar with paper. 

**Overall, Our portfolios sorted by gross profit/asset shows the same pattern as portfolios in the original paper.**

In [39]:
#return for each bins of BM
portfolios_bm = calcu_return(t4_df_2,'bin_bm')

In [45]:
#L & S market-to-book strategy and Regression 
regre_bm = ff_regression(portfolios_bm,'bin_bm', 5)
print('Fama-French Regression:')
regre_bm
#characterstic of BM
characteristics_bm = chara_sort(t4_df_2,'bin_bm')
print('Time-Series Average Portfolio Characteristics')
characteristics_bm

Fama-French Regression:


,group,re,alpha,MKT,SMB,HML
0,1.0,1.051141,0.005232,0.994116,-0.115531,-0.344812
1,1,,[11.604106],[95.357288],[-7.567749],[-22.594468]
2,2.0,1.010314,0.003649,0.991017,-0.095926,0.06702
3,2,,[6.692105],[78.603788],[-5.195783],[3.631364]
4,3.0,1.022707,0.003069,0.987655,-0.061779,0.309436
5,3,,[5.082791],[70.740396],[-3.021700],[15.140346]
6,4.0,1.079488,0.002689,1.009973,0.021483,0.562732
7,4,,[4.451310],[72.303607],[1.050239],[27.520427]
8,5.0,1.258536,0.003055,1.089148,0.183915,0.82797
9,5,,[3.631473],[55.987623],[6.456124],[29.075218]


Time-Series Average Portfolio Characteristics


,GPA,BM,ME,number of firm
bin_bm,,,,
1,0.391795,0.236033,5555.872621,622.332790
2,0.230061,0.480432,3420.481266,509.184339
3,0.147266,0.695335,2250.711991,504.339315
4,0.109684,0.943925,1781.571118,543.598695
5,0.097079,122.009107,1078.231514,633.130506


### Replication
<center>Paper</center> 
<img src="dataset/t4_panel_b_paper.jpg" alt="paper table 4 panel B" width=60%/>

**Interpretation：**

*The regression shows a different pattern with paper.*

1. A univariate sort on book-to-market yields a value portfolio “polluted” with unprofitable stocks, and a growth portfolio “polluted” with profitable stocks. A value strategy that avoids holding stocks that are “more unprofitable than cheap,” and avoids selling stocks that are “more profitable than expensive,” should outperform conventional value strategies. Similarly, a profitability strategy that avoids holding stocks that are profitable but “fully priced,” and avoids selling stocks that are unprofitable but “cheap,” should outperform conventional profitability strategies.

# Table 5

### Size portfolio time series average characteristics.

This table reports the **time series averages** of the characteristics of **quintile portfolios** sorted on **market equity**. Portfolio breakpoints are based on **NYSE stocks only**.

The sample excludes financial firms (those with one-digit standard industrial classification codes of six) and covers July 1963 to December 2010.

In [46]:
# Read cleaned data
t5_df = t2_df
#first group by date
t5_df_groups = t5_df.groupby('date')

#create empty dataframe to store sorted data
desired_columns = list(t5_df.columns)+['bin_mktcap']
df_sort_on_size = pd.DataFrame(columns = desired_columns)

for date in tqdm(set(t5_df.date)):
    group = t5_df_groups.get_group(date)
    group = nyseBreakpoints(group, 'mktcap').reset_index()[desired_columns] #sort by market cap 
    df_sort_on_size = df_sort_on_size.append(group, ignore_index = True)

100%|█████████████████████████████████████████| 613/613 [02:12<00:00,  4.64it/s]


In [58]:
# report time series average characteristics of the size portfolios
characteristics_size_portfolio = (
    df_sort_on_size[
        df_sort_on_size['bin_mktcap'].notnull() &
        df_sort_on_size['bm'].notnull() &
        df_sort_on_size['mktcap'].notnull()&
        df_sort_on_size['gp_at'].notnull()
    ]
    .groupby(['date'])[['mktcap', 'bm','gp','permno','at', 'bin_mktcap', 'date']]
)

def characteristicByDate(g_d):
    g_d['number of firms each day'] = g_d['permno'].nunique() #number of firms each day
    g_d['total mktcap each day'] = g_d['mktcap'].sum() #total market cap each day
    
    g_d = g_d.groupby(['bin_mktcap']) #in each group of day, group by size
    
    desired_columns = ['date', 'bin_mktcap', 'number of firms', 'percent of firms', 'average capitalization (millions of dollars)', \
    'total capitalization (billions of dollars)', 'total capitalization (percent)', 'portfolio book-to-market', \
        'portfolio gross profits-to-assets']
    characteristics = pd.DataFrame(columns = desired_columns)
    
    for size in range(1, 6):
        try:
            g_s = g_d.get_group(size)
            g_s['number of firms'] = float(g_s['permno'].nunique())
            g_s['percent of firms'] = g_s['permno'].nunique()/g_s['number of firms each day']*100
            g_s['average capitalization (millions of dollars)'] = g_s['mktcap'].mean()
            g_s['total capitalization (billions of dollars)'] = g_s['mktcap'].sum()/1000
            g_s['total capitalization (percent)'] = g_s['mktcap'].sum()/g_s['total mktcap each day']*100
            g_s['portfolio book-to-market'] = (g_s['mktcap']*g_s['bm']).sum()/g_s['total mktcap each day']
            g_s['portfolio gross profits-to-assets'] = g_s['gp'].sum()/g_s['at'].sum()   

            g_s = g_s[desired_columns]    
        
        # if no value in the group, set all characteristics value equal to 0 
        except:
            g_s = pd.DataFrame(columns = desired_columns)
            for col in g_s.columns:
                g_s[col].values[:] = 0
            
        characteristics = characteristics.append(g_s, ignore_index = True)
   
    # group by size again to calculate the time series average value        
    characteristics_d = characteristics.groupby(['bin_mktcap']).mean().reset_index(drop=False)

    return characteristics_d

desired_columns = ['bin_mktcap', 'number of firms', 'percent of firms', 'average capitalization (millions of dollars)', \
    'total capitalization (billions of dollars)', 'total capitalization (percent)', 'portfolio book-to-market', \
        'portfolio gross profits-to-assets']
characteristics_s = pd.DataFrame(columns = desired_columns)
            
for date in tqdm(set(df_sort_on_size.date)):
    group = characteristics_size_portfolio.get_group(date)
    characteristics_d = characteristicByDate(group).reset_index()[desired_columns]
    characteristics_s = characteristics_s.append(characteristics_d, ignore_index = True)

characteristics_panel = characteristics_s.groupby(['bin_mktcap']).mean()

#report the results
characteristics_panel

100%|██████████| 613/613 [00:12<00:00, 49.58it/s]


,number of firms,percent of firms,average capitalization (millions of dollars),total capitalization (billions of dollars),total capitalization (percent),portfolio book-to-market,portfolio gross profits-to-assets
bin_mktcap,,,,,,,
1,1356.122349,47.158480,99.217823,153.980787,1.838947,0.016064,0.209932
2,486.443719,17.358741,503.604981,269.778099,3.052098,0.021881,0.168726
3,367.859706,13.402833,1179.431190,456.649727,5.540094,0.037978,0.148193
4,309.432300,11.355870,2929.474379,964.637330,11.867650,0.078332,0.145372
5,290.125612,10.724076,22392.099788,6928.125025,77.701211,6.033188,0.159690


### Replication
<center>Paper</center> 
<img src="dataset/t5_paper.jpg" alt="paper table 5" width=60%/>

**Interpretation：**

1. More than half of firms are in the nano-cap portfolio, but these stocks make up less than 2% of the market by capitalization, 
2. The large cap portfolio typically contains fewer than 300 stocks, but makes up roughly three-quarters of the market by capitalization. 
3. The portfolios exhibit a great deal of variation in book-to-market, with the smaller stocks tending toward value and the larger stocks toward growth. **Compared to the pattern shown in original paper, the variation here is even larger, with the large stocks group shows a time-series average B/M ratio greater than 6.** 
4. **Compared to the original paper, out portfolios exhibit more variations in profitability.**

# Table 6

### Double sorts on gross profits-to-assets and market equity.

This table shows the **value-weighted average excess returns to portfolios double sorted**, using NYSE breakpoints, on **gross profits-to-assets and market equity**, and results of **time series regressions of both sorts’ high minus low portfolios’ returns on the Fama and French factors [the market, size and value factors MKT, SMB (small-minus-large), and HML (high-minus-low)]**. Test statistics are given in square brackets. 

The table also shows **the average number of firms in each portfolio and each portfolios’ average book-to-market** (the portfolios exhibit little gross-profits to asset variation within size quintiles and little size variation within profitability quintiles). 

The sample excludes financial firms (those with one-digits tandard industrial classification codes of six) and covers December 1970 to December 2021.

In [67]:
#df_sort_on_size = pd.read_parquet(path + os.sep + 'df_sort_on_size.parquet')
df_sort_on_size.columns

Index(['cusip', 'gvkey', 'permno', 'date', 'bm', 'mktcap', 'ret', 'price',
       'ffi49', 'log_bm', 'log_size', 'ret_1_0', 'mktcap_lag1', 'ret_12_2',
       'fyear', 'at', 'gp', 'ib', 'fcf', 'exchg', 'gp_at', 'ib_at', 'fcf_at',
       'gp_at_dm', 'ib_at_dm', 'fcf_at_dm', 'gp_at_w', 'ib_at_w', 'fcf_at_w',
       'gp_at_dm_w', 'ib_at_dm_w', 'fcf_at_dm_w', 'log_bm_w', 'log_size_w',
       'ret_1_0_w', 'ret_12_2_w', 'bin_mktcap'],
      dtype='object')

In [47]:
#Panel A: Portfolio average excess returns and time series regression results
#value-weighted average excess returns to portfolios double sorted
#create full_data with double sorted
t6_df_sorts = pd.merge(
            t4_df_2,
            df_sort_on_size[['gvkey','date','bin_mktcap']],
            on = ['date','gvkey']
            )

In [69]:
t6_df_sorts .columns

Index(['index', 'cusip', 'gvkey', 'permno', 'date', 'bm', 'mktcap', 'ret',
       'log_bm', 'log_size', 'ret_1_0', 'mktcap_lag1', 'ret_12_2', 'exchg',
       'gp_at', 'ib_at', 'fcf_at', 'gp', 'at', 'gp_at_lag12', 'rf', 'ret_rf',
       'bin_gp_at', 'bin_bm', 'bin_mktcap'],
      dtype='object')

In [48]:
r_2sort = (
    t6_df_sorts[
        t6_df_sorts['bin_mktcap'].notnull() &
        t6_df_sorts['ret'].notnull() &
        t6_df_sorts['mktcap'].notnull()
    ]
    .groupby(['bin_mktcap', 'bin_gp_at'])[['ret_rf', 'mktcap_lag1']]
    .apply(
        lambda g: pd.Series({
            'vw':(g['ret_rf'] * g['mktcap_lag1']).sum() / g['mktcap_lag1'].sum() 
        })
    )
    .reset_index()
)

table_r_2sort = r_2sort.set_index(['bin_gp_at','bin_mktcap'])
table_r_2sort = table_r_2sort.unstack().reset_index()
table_r_2sort 

bin_gp_at        vw                                        
bin_mktcap                   1         2         3         4         5
0                  1 -0.009189 -0.000285  0.003255  0.005218  0.006435
1                  2 -0.012677 -0.005006  0.001107  0.003384  0.006418
2                  3 -0.011578 -0.000617  0.003731  0.006681  0.009141
3                  4 -0.007222  0.002006  0.006442  0.008080  0.011299
4                  5 -0.004915  0.002339  0.006822  0.008888  0.012832

In [65]:
# create re
diff_r = pd.DataFrame(table_r_2sort.iloc[:,-1] - table_r_2sort.iloc[:,1])
diff_r.index = range(0, 2*len(diff_r), 2)
diff_r = diff_r.reindex(index=range(2*len(diff_r)))


diff_r_cap = pd.DataFrame((table_r_2sort.iloc[-1,:] - table_r_2sort.iloc[0,:])[1:])
diff_r_cap.index = range(0, 2*len(diff_r_cap), 2)
diff_r_cap = diff_r_cap.reindex(index=range(2*len(diff_r_cap)))

In [66]:
r_2sort = (
    t6_df_sorts[
        t6_df_sorts['bin_mktcap'].notnull() &
        t6_df_sorts['ret'].notnull() &
        t6_df_sorts['mktcap'].notnull()
    ]
    .groupby(['bin_mktcap', 'bin_gp_at','date'])[['ret_rf', 'mktcap_lag1']]
    .apply(
        lambda g: pd.Series({
            'vw': (g['ret_rf'] * g['mktcap_lag1']).sum() / g['mktcap_lag1'].sum() 
        })
    )
    .reset_index()
)

#smb strategy and Regression 
grouped = r_2sort .groupby('bin_mktcap')
result = pd.DataFrame()
for i in range(1,6):
    #L & S strategy
    tem_portfolio = grouped.get_group(i)
    tem_portfolio = pd.merge(
        tem_portfolio.query('bin_gp_at==5'),
        tem_portfolio.query('bin_gp_at==1'),
        suffixes=['_short', '_long'],
        on='date'
)
    tem_portfolio['strategy_vw_mktcap'] = tem_portfolio['vw_long'] - tem_portfolio['vw_short']
    #Regression
    tem_portfolio = pd.merge(
        tem_portfolio,
        ff4,
        on='date' )
    l1 = smf.ols('strategy_vw_mktcap ~  1 + mkt_rf + hml + smb', data = tem_portfolio ).fit()
    #print(sm.iolib.summary2.summary_col([l1], stars=True))
    #tem = pd.DataFrame(np.insert(l1.params.values,0,np.average(tem_portfolio['portfolio_vw']))).T
    result = result.append(pd.DataFrame(l1.params.values).T, ignore_index=True)
    #tem = pd.DataFrame(np.insert(l1.tvalues.values,0,np.average(tem_portfolio['portfolio_vw']))).T
    result = result.append(pd.DataFrame(['['+str(format(x, '.6f'))+']' for x in l1.tvalues.values]).T, ignore_index=True)
result.columns = ['alpha','MKT','SMB','HML']
result['re']=diff_r * 100
result

,alpha,MKT,SMB,HML,re
0,-0.005975,-0.143689,0.213917,-0.158987,1.562400
1,[-5.310734],[-5.523608],[5.617567],[-4.173600],NaN
2,-0.004162,-0.089246,0.378914,-0.158594,1.909508
3,[-3.463547],[-3.211573],[9.314765],[-3.897318],NaN
4,-0.004327,-0.086733,0.493106,-0.150995,2.071856
5,[-3.467838],[-3.006121],[11.675241],[-3.573840],NaN
6,-0.004655,0.024022,0.503162,-0.105935,1.852129
7,[-3.833690],[0.855664],[12.243405],[-2.576806],NaN
8,-0.005898,0.244898,0.835534,0.057373,1.774739
9,[-4.824082],[8.663086],[20.190887],[1.385932],NaN


In [67]:
#profitability strategy and Regression 
grouped = r_2sort .groupby('bin_gp_at')
result = pd.DataFrame()
for i in range(1,6):
    #L & S strategy
    tem_portfolio = grouped.get_group(i)
    tem_portfolio = pd.merge(
        tem_portfolio.query('bin_mktcap==5'),
        tem_portfolio.query('bin_mktcap==1'),
        suffixes=['_short', '_long'],
        on='date'
)
    tem_portfolio['strategy_vw_gp_at'] = tem_portfolio['vw_long'] - tem_portfolio['vw_short']
    #Regression
    tem_portfolio = pd.merge(
        tem_portfolio,
        ff4,
        on='date' )
    l1 = smf.ols('strategy_vw_gp_at ~  1 + mkt_rf + hml + smb', data = tem_portfolio ).fit()
    #print(sm.iolib.summary2.summary_col([l1], stars=True))
    #tem = pd.DataFrame(np.insert(l1.params.values,0,np.average(tem_portfolio['portfolio_vw']))).T
    result = result.append(pd.DataFrame(l1.params.values).T, ignore_index=True)
    #tem = pd.DataFrame(np.insert(l1.tvalues.values,0,np.average(tem_portfolio['portfolio_vw']))).T
    result = result.append(pd.DataFrame(['['+str(format(x, '.6f'))+']' for x in l1.tvalues.values]).T, ignore_index=True)
result.columns = ['alpha','MKT','SMB','HML']
result['re']=diff_r_cap * 100
result

,alpha,MKT,SMB,HML,re
0,-0.015157,-0.220537,-0.027672,1.111153,0.427373
1,[-12.140720],[-7.639629],[-0.654845],[26.285446],NaN
2,-0.019408,0.145604,0.455505,1.121705,0.262338
3,[-17.475708],[5.669857],[12.117049],[29.828336],NaN
4,-0.019018,0.134075,0.475799,1.26736,0.356649
5,[-16.538177],[5.042316],[12.223890],[32.548622],NaN
6,-0.017089,0.116246,0.651198,1.220301,0.366965
7,[-15.837054],[4.658907],[17.828872],[33.398346],NaN
8,-0.01508,0.16805,0.593945,1.327512,0.639712
9,[-12.925146],[6.229167],[15.039789],[33.603204],NaN


### Replication
<center>Paper</center> 
<img src="dataset/t6_panel_a_paper.jpg" alt="paper table 6 panel A" width=60%/>

**Interpretation:**

*Portfolios are formed by independently quintile sorting on the two variables, using NYSE breaks. The sample excludes financial firms, and covers July 1963 to December 2009. The re is the gap between maxmimum return and minmimum return amoing each bin, the t value of re is filled with NA. For the regression table, even rows are regression coefficients, odd rows are T values. Each two lines represents a bin.*

1. Similar with paper, the portfolios exhibit little variation in gross profits-toassets within profitability quintiles, and little variation in size within size quintiles. 
2. Profitability spread is large and significant across size quintiles. The spreads are decreasing across size quintiles.



In [71]:
#panel B: Portfolio average number of firms and portfolio book-to-markets
#Number of firms

# groupby date
t6_df_sorts_gp = t6_df_sorts[
        t6_df_sorts['bin_mktcap'].notnull() &
        t6_df_sorts['ret'].notnull() &
        t6_df_sorts['mktcap_lag1'].notnull()&
        t6_df_sorts['gp_at'].notnull()
    ].groupby('date')

key1 = list(t6_df_sorts_gp.groups.keys())[0]

t6b = (
    t6_df_sorts_gp.get_group(key1).reset_index()
    .groupby(['bin_mktcap', 'bin_gp_at'])[['permno']]
    .apply(
        lambda g: pd.Series({
            'num_fimm':g['permno'].nunique()
        })
    )
    .reset_index()
)
t6b = t6b.set_index(['bin_gp_at','bin_mktcap'])
t6b = t6b.unstack().reset_index()

for i in list(t6_df_sorts_gp.groups.keys())[1:]:
    new_df = (
        t6_df_sorts_gp.get_group(i).reset_index()
        .groupby(['bin_mktcap', 'bin_gp_at'])[['permno']]
        .apply(
            lambda g: pd.Series({
                'num_fimm':g['permno'].nunique()
            })
        )
        .reset_index()
    )
    new_df = new_df.set_index(['bin_gp_at','bin_mktcap'])
    new_df = new_df.unstack().reset_index()
    t6b = pd.concat([t6b, new_df], axis = 0)

t6b_2 = t6b.groupby(t6b.index).agg('mean')
t6b_2

bin_gp_at    num_fimm                                             
bin_mktcap                     1           2          3          4          5
0                1.0  374.947798  108.636215  84.727569  65.717781  61.845024
1                2.0  159.841762   72.225122  61.893964  60.296900  57.246330
2                3.0  209.712887   86.869494  68.396411  58.928222  54.685155
3                4.0  267.323002  104.042414  71.187602  59.908646  54.887439
4                5.0  344.898858  114.926591  81.766721  65.003263  61.549755

In [73]:
# groupby date
t6_df_sorts_gp = t6_df_sorts[
        t6_df_sorts['bin_mktcap'].notnull() &
        t6_df_sorts['ret'].notnull() &
        t6_df_sorts['mktcap_lag1'].notnull()&
        t6_df_sorts['gp_at'].notnull()
    ].groupby('date')

key1 = list(t6_df_sorts_gp.groups.keys())[0]

t6bbm = (t6_df_sorts_gp.get_group(key1).reset_index()
         .groupby(['bin_mktcap', 'bin_gp_at'])[['mktcap','bm']]
         .apply(
         lambda g: pd.Series({
            'port_bm': (g['mktcap']*g['bm']).sum()/g['mktcap'].sum()
        })
    )
    .reset_index()
)

t6bbm = t6bbm.set_index(['bin_gp_at','bin_mktcap'])
t6bbm = t6bbm.unstack().reset_index()

for i in list(t6_df_sorts_gp.groups.keys())[1:]:
    new_df = (
         t6_df_sorts_gp.get_group(i).reset_index()
         .groupby(['bin_mktcap', 'bin_gp_at'])[['mktcap','bm']]
         .apply(
         lambda g: pd.Series({
            'port_bm': (g['mktcap']*g['bm']).sum()/g['mktcap'].sum()
        })
    )
    .reset_index()
    )
    new_df = new_df.set_index(['bin_gp_at','bin_mktcap'])
    new_df = new_df.unstack().reset_index()
    t6bbm = pd.concat([t6bbm, new_df], axis = 0)

t6b_2bm = t6bbm.groupby(t6bbm.index).agg('mean')
t6b_2bm

bin_gp_at   port_bm                                         
bin_mktcap                   1         2         3         4          5
0                1.0  1.048353  0.952733  0.920127  0.946058  29.943619
1                2.0  1.354950  0.908221  0.871552  0.804709   0.781076
2                3.0  1.121296  0.763765  0.656236  0.635108   0.563713
3                4.0  0.821113  0.637744  0.564097  0.490970   0.392519
4                5.0  0.712760  0.529614  0.431104  0.353227   0.248483

### Replication
<center>Paper</center> 
<img src="dataset/t6_panel_b_paper.jpg" alt="paper table 6 panel B" width=60%/>

**Charectersitic:**

1. With profitability growing, the number of company increases. The variation is obvious
2. Little variation in size within size quintiles
3. With profitability growing, the BM ratio has a decreasing trend, but there are some exceptions in small size companies. 
4. With size growing, the BM ratio has a decreasing trend, but there are some exceptions in extreme large(or samll) profitability companies. 
**Overall, Our portfolios sorted by gross profit/asset & size shows the same pattern as portfolios in the original paper.**

# Table 7

## Performance of large stock profitability and value strategies.

*Panel A*: This table shows the performance of portfolios for **medusing only the 500 largest non financial firms** for which **gross profits-to-assets (GP/A) and book-to-market (B/M) are both available**, and results of time series regressions of these portfolios’ returns on the Fama and French factors [the market factor (MKT), the size factor small-minus-large(SMB), and the value factor high-minus-low (HML)], with test-statistics (in square brackets). Portfolios are tertile sorted on GP/A (PanelA).The table also shows **time series average portfolio characteristics** [portfolio GP/A, portfolio B/M, average firm size (ME, in billions of dollars), and number of firms(n)]. 

The sample covers **December 1970 to December 2021**.

In [75]:
# Read cleaned data
t7_df = pd.read_parquet(path + os.sep + 'gp.parquet')

t7_df = pd.merge(
    t7_df,
    ff4[['date','rf']],
    on = 'date')

#create excess return - ret_rf
t7_df['ret_rf'] = t7_df['ret'] - t7_df['rf']
#create a month column for rebalancing at yearly frequency
t7_df['month'] = round(t7_df.date/100)%1000
t7_df['fyear'] = round(t7_df.date/10000)

#get information last June
t7_df['mktcap_lag13'] = t7_df.sort_values(['date']).groupby('permno')['mktcap'].shift(periods = 13)
t7_df['price_lag12'] = t7_df.sort_values(['date']).groupby('permno')['price'].shift(periods = 12)

#only keep data in June each year
t7_df = t7_df[(
    (t7_df.month == 6) & #stock info in June
    t7_df.mktcap_lag13.notnull() &
    t7_df.price_lag12.notnull()
)]

In [76]:
#medusing 500 largest stocks
t7_df_groups = t7_df.groupby('date')

columns = list(t7_df.columns)
fortune500 = pd.DataFrame(columns = columns)

for date in set(t7_df.date):
    group = t7_df_groups.get_group(date).sort_values('mktcap_lag13')[-500:]
    fortune500 = fortune500.append(group, ignore_index = True)

In [77]:
fortune500 = fortune500.sort_values(['permno','date'])

# Create bins
fortune500['rank'] = (
  fortune500
  .groupby('date')
  .apply(lambda group: group.gp_at.rank())
    #we don't use NYSE breakpoints here because the sample size is smaller now
).reset_index(level=[0], drop=True).sort_index()

fortune500['bin_gp_at'] = fortune500.apply(lambda x : 3 if (x['rank'] > 350) else (1 if (x['rank'] <= 150) else 2), axis=1)

In [78]:
#calculated equal weighted return for each bin
fortune500_gp = (
    fortune500
    .groupby(['date', 'bin_gp_at']) 
    .apply(
        lambda g: pd.Series({
            'portfolio_vw': ((g['price']-g['price_lag12'])/g['price_lag12']).mean() #equal-weighted portfolio
      })
    )).reset_index()

fortune500_gp = fortune500_gp.sort_values(['date', 'bin_gp_at'])

In [79]:
regression = ff_regression(fortune500_gp, 'bin_gp_at', 3, t7=True)
print('Fama-French Regression:')
regression

characteristics = chara_sort(fortune500, 'bin_gp_at')
print('Time-Series Average Portfolio Characteristics')
characteristics

Fama-French Regression:


,group,re,alpha,MKT,SMB,HML
0,1.0,-0.100464,-0.134633,4.323915,-9.732186,-12.560258
1,1,,[-1.302772],[0.859448],[-1.980775],[-1.953657]
2,2.0,0.319859,-0.038987,5.313635,-6.71007,-7.950454
3,2,,[-1.388030],[3.885948],[-5.024752],[-4.549929]
4,3.0,0.792791,0.157788,-1.057869,-0.90453,0.728425
5,3,,[1.205354],[-0.165996],[-0.145335],[0.089445]
6,H-L,0.893255,0.29242,-5.381784,8.827656,13.288683


Time-Series Average Portfolio Characteristics


,GPA,BM,ME,number of firm
bin_gp_at,,,,
1,0.050282,0.953060,12707.995134,149.2
2,0.250375,0.526984,19228.472326,199.4
3,0.546400,0.304125,25261.657738,150.0


### Replication
<center>Paper</center> 
<img src="dataset/t7_panel_a_paper.jpg" alt="paper table 7 panel A" width=60%/>

### Interpretation

The strategy is constructed within the 500 largest non-financial stocks for which gross profits-to-assets and book-to-market are both available. Each year I rank these stocks based on their gross profits-to-assets. At the end of each June the strategy buys one dollar of each of the 150 stocks with the highest average of the profitability and value ranks, and shorts one dollar of each of the 150 stocks with the lowest average ranks. The performance of this strategy is provided in Table 7. The table also shows, for comparison, the performance of similarly constructed strategies based on profitability and value individually.